
# 4.3.3 htmファイルからMD＆A情報としてみなされる文章のみを抽出 

　htmファイルから、 BeautifulSoupを用いて MD＆A情報としてみなされる文章のみを抽出した。BeautifulSoupとは、Pythonのライブラリのひとつで、htmlやxmlといったWebページで用いられるデータを構文解析する際に用いられるライブラリである。今回は、 htm形式のテキストファイルからMD&A部分を抜き出し、htm形式のタグの除去等の処理を行って生の文章を抽出した。また文章を抽出においては、加藤・五島(2020)を参考に「１【経営方針、経営環境及び対処すべき課題等】」と「３【経営者による財政状態、経営成績及びキャッシュ・フローの状況の分析】」の 2 つの章のテキストをMD&Aの記述とみなして抽出を行った。

In [6]:
from datetime import datetime as dt
import glob
import pandas as pd
import bs4
import re
from bs4 import BeautifulSoup
import os


def fetch_md_and_a(data_frame_name, data_frame):

    htm_files = call_htm_files(data_frame_name)
    industry_htm_files_hash = make_industry_htm_files_hash(htm_files , data_frame_name, data_frame)
    
    sum_csv_output = 1
    for industry_name, htm_files in industry_htm_files_hash.items():
        
        edinet_id_files_hash = {}
        #Edninetの識別子ごとでファイル群を管理する
        for file in htm_files:
            pattern = 'E.*?-'
            result = re.search(pattern, file)
            start, end = result.span()
            identifier = file[start: end-1]

            if identifier in edinet_id_files_hash :
                edinet_id_files_hash[identifier].append(file)
            else:
                edinet_id_files_hash[identifier] = [file]
        
        idx_num = 1
        # すでにEdninetの識別子順でソートされている
        for identifier, e_files in edinet_id_files_hash.items() :
            # 「１【経営方針、経営環境及び対処すべき課題等】」,「３【経営者による財政状態、経営成績及びキャッシュ・フローの状況の分析】」            
            #コロナ前, コロナ過渡期, コロナ後の最低３つがなかった時
#             if len(e_files) < 3:
#                 print(f"detected: {e_files}")
#                 continue
            
            date_files_hash = {}
            for e_f in e_files:
                date_str = get_date_str_val(e_f)
                if date_str in date_files_hash :
                    date_files_hash[date_str].append(e_f)
                else:
                    date_files_hash[date_str] = [e_f]
            
            for date_str, d_files in date_files_hash.items():
                
                d_files_len = len(d_files)

                if d_files_len > 1:
                    print(f"count: {d_files_len}, {d_files}")
                
                md_a_df = pd.DataFrame()
                idx = 0
                while  idx < d_files_len :
                    sp = read_file(d_files[idx])
                    htm_contents = sp.find('body').div.contents
                    md_a_contents, func =  extract_md_a_contents(htm_contents)
                    tmp_df = func(md_a_contents)                
                    md_a_df  = pd.concat([md_a_df, tmp_df], axis=0)
                    idx += 1

                save_md_a_df_to_csv(md_a_df, idx_num, date_str , data_frame_name,  industry_name)
                #beforeとafterでユニークなindex_numを持つようにする
                idx_num += 1
                
                print(sum_csv_output, len(md_a_df))
                sum_csv_output += 1

In [7]:
def call_htm_files(data_frame_name):
    htm_files = glob.glob(f"/home/jovyan/2UnzippingHtm/UnzipedHtmFiles/{data_frame_name}/**/*.htm", recursive=True)
    return  htm_files


def make_industry_htm_files_hash(htm_files , data_frame_name: str, data_frame):
    industry_htm_files_hash = {}
    
    industry_list = make_type_of_industry_list(data_frame)
    for industry in industry_list:
        industry_htm_files_hash[industry]= list(filter(lambda x:  industry in x , htm_files))
    
    return  industry_htm_files_hash 


def make_type_of_industry_list(data_frame : pd.DataFrame, industry_col="[業種（東証）]"):
    return ["サービス業"]


def get_date_str_val(htm_file) :
    date_str = htm_file[-20:-10]
    return  date_str


def read_file(htm_path: str) -> BeautifulSoup:
    with open(htm_path, mode="r") as f:
        htm = f.read()
        sp = BeautifulSoup(htm, "html.parser")
        return sp

"""
htmファイルには3つのパターンが存在しているため、その都度ハンドリングをする


1. smt_textパターン: 属性値にsmt_textを持っているhtm
2. textjustify_spanパターン: smt_textを持たず、spanにテキスト情報を記載しているhtm
3. text_indent?
"""
#-----------------------------------------------------
#md_aの章番号を指定
#企業によって全角と半角かわかれる可能性があるためどちらも判定
# 「１【経営方針、経営環境及び対処すべき課題等】」,「３【経営者による財政状態、経営成績及びキャッシュ・フローの状況の分析】」
# after_2017 = ['１', '1','３', '3']

def extract_md_a_contents(htm_contents: list, chapter_numbers=['１', '1','３', '3']) -> tuple:
    
    #MD＆A情報を持つ章を抽出
    md_a_contents = []
    
    for content in htm_contents :
        #bs4.element.Tag型ではないとfindメソッドでtypeエラーを起こすため判定
        if not isinstance(content, bs4.element.Tag) :
            continue
        
        found = content.find(class_=re.compile("smt_head"))
        #findしたけどデータがない(s4.element.Tag型ではない)がない場合.textプロパティを呼び出す際にtypeエラーを起こすため判定
        if not  found:
            found = content.find("h3")
            if not  found:
                continue
            else: 
                func = insert_df_by_span_p
        else :
            func = insert_df_by_smt
    
        title = found.text
        #md_aの章番号を指定
            # "1【経営方針、経営環境及び対処すべき課題等】",
            # "3【経営者による財政状態、経営成績及びキャッシュ・フローの状況の分析】"
        #企業によって全角と半角か別れる可能性があるためどちらも判定
        if title[0] in  chapter_numbers :
             md_a_contents.append(content)
            
    return md_a_contents, func

#------------------------------------------
def insert_df_by_span_p(md_a_contents: list) -> pd.DataFrame:
    
    md_a_df = pd.DataFrame()

    text_values = []
    for content in  md_a_contents : 
        
        # テキスト情報を持つbs4.element.Tag型のみ取得
        found_values = content.find_all("p")
         # テキストを取得
      
        for val in found_values:
            if val.parent.name in ["td" , "tr"] or val.parent.parent.name in ["td" , "tr"]:
                continue
    
            if val.span != None:
                text = val.span.text
                if len(text) > 1:
                    text_values.append(text)
            else :
                text = val.text
                if len(text) > 1:
                    text_values.append(text)
     
        tmp_df = pd.DataFrame()
        tmp_df['Text'] = text_values 
        #pandas.DataFrame型に入れる
        md_a_df  = pd.concat([md_a_df ,  tmp_df], axis=0)
        
    return md_a_df

#-----------------------------------------------------
def insert_df_by_smt(md_a_contents: list) -> pd.DataFrame:
    
    md_a_df = pd.DataFrame()

    for content in  md_a_contents : 
        # テキスト情報を持つbs4.element.Tag型のみ取得
        found_values = content.find_all(class_=re.compile("smt_text"))
         # テキストを取得
        text_values = list(filter(lambda val: len(val) > 1, list(map(lambda val: val.text,found_values))))
        tmp_df = pd.DataFrame()
        tmp_df['Text'] = text_values 
        #pandas.DataFrame型に入れる
        md_a_df  = pd.concat([md_a_df ,  tmp_df], axis=0)
        
    return md_a_df

#----------------------------------------------
        
def save_md_a_df_to_csv(md_a_df: pd.DataFrame, idx_num: int, date_str: str,  data_frame_name, industry_name) -> None :
    #Sampleフォルダの作成
    
    
    filepath = os.getcwd()+ f"/{data_frame_name}"
    if not os.path.exists(filepath) :
        os.mkdir(filepath)
        
    filepath = os.getcwd()+ f"/{data_frame_name}/{industry_name}"
    if not os.path.exists(filepath) :
        os.mkdir(filepath)
    
    filepath_before_pandemic  = os.getcwd()+ f"/{data_frame_name}/{industry_name}"  +  "/BeforeSample"
    if not os.path.exists(filepath_before_pandemic) :
        os.mkdir(filepath_before_pandemic)
    
    filepath_transition_period_pandemic = os.getcwd()+ f"/{data_frame_name}/{industry_name}"  +  "/TransitionPeriodSample"
    if not os.path.exists(filepath_transition_period_pandemic) :
        os.mkdir(filepath_transition_period_pandemic)
    
    filepath_after_pandemic = os.getcwd()+ f"/{data_frame_name}/{industry_name}"  +  "/AfterSample"
    if not os.path.exists(filepath_after_pandemic) :
        os.mkdir(filepath_after_pandemic)
            
    
    date_dt = dt.strptime(date_str, '%Y-%m-%d')
    
    before_pandemic_boundary_val = dt(2019, 3, 31)
    
    transition_period_pandemic_boundary_val = dt(2020, 3, 31)
    if  date_dt <= before_pandemic_boundary_val :
        md_a_df.to_csv(filepath_before_pandemic+"/"+f'{idx_num}_{date_str}.csv')
    elif  date_dt <= transition_period_pandemic_boundary_val :
        md_a_df.to_csv(filepath_transition_period_pandemic +"/"+f'{idx_num}_{date_str}.csv')
    else :
        md_a_df.to_csv(filepath_after_pandemic+"/"+f'{idx_num}_{date_str}.csv')

In [8]:
import pandas as pd

data_frame_name ="top_20" 
data_frame = pd.read_csv("/home/jovyan/1CallingEdinetApi"+f"/EdinetIdxFiles/{data_frame_name}.csv", skiprows=4)

In [9]:
fetch_md_and_a(data_frame_name, data_frame)

1 183
2 81
3 123
4 133
5 129
6 117
7 211
8 107
9 73
10 101
11 82
12 137
13 123
14 172
15 125
16 120
17 165
18 73
19 135
20 124
21 118
22 46
23 203
24 68
25 66
26 74
27 65
28 164
29 115
30 139
31 112
32 57
33 132
34 156
35 91
36 58
37 171
38 65
39 213
40 108
41 102
42 322
43 176
44 183
45 81
46 123
47 133
48 129
49 117
50 211
51 107
52 73
53 101
54 82
55 137
56 123
57 172
58 125
59 120
60 165
61 73
62 135
63 124
64 118
65 46
66 203
67 68
68 66
69 74
70 65
71 164
72 115
73 139
74 112
75 57
76 132
77 156
78 91
79 58
80 171
81 65
82 213
83 108
84 102
85 322
86 176
87 183
88 81
89 123
90 133
91 129
92 117
93 211
94 107
95 73
96 101
97 82
98 137
99 123
100 172
101 125
102 120
103 165
104 73
105 135
106 124
107 118
108 46
109 203
110 68
111 66
112 74
113 65
114 164
115 115
116 139
117 112
118 57
119 132
120 156
121 91
122 58
123 171
124 65
125 213
126 108
127 102
128 322
129 176
130 183
131 81
132 123
133 133
134 129
135 117
136 211
137 107
138 73
139 101
140 82
141 137
142 123
143 172
144 12